In [ ]:
!pip install timm
!pip install torchvision
!pip install shap
!git clone https://github.com/MadryLab/robustness
!pip install ./robustness

In [ ]:
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn
import torch
from robustness import model_utils, datasets, train, defaults
import timm
from cox.utils import Parameters
import cox.store
from google.colab import drive
drive.mount('/content/gdrive')

ds = datasets.CIFAR(data_path='./data')

model = timm.create_model('ecaresnet26t', num_classes=10, pretrained=True)
m, _ = model_utils.make_and_restore_model(arch=model,resume_path="/content/gdrive/My Drive/Final Adversarial CIFAR10/checkpoint.pt.best", dataset=ds, add_custom_forward=True)
train_loader, val_loader = ds.make_loaders(batch_size=20, workers=4)

# Create a cox store for logging
out_store = cox.store.Store("/content/gdrive/My Drive")

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'lr': 0.001,
    'save_ckpt_iters': 1,
    'attack_lr': 1.5, 
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, datasets.CIFAR)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, datasets.CIFAR)
# Train a model
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)

In [ ]:
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn
import torch
from robustness import model_utils, datasets, train, defaults
import timm
from cox.utils import Parameters
import cox.store
from google.colab import drive
import torchvision.transforms as transforms
import torchvision.models as models

transform_train = transforms.Compose(
    [transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10), transforms.ToTensor()])
drive.mount('/content/gdrive')
transforms_kwargs = {
        "transform_train":transform_train
}
ds = datasets.CIFAR(data_path='./data', **transforms_kwargs)

model = timm.create_model('ecaresnet26t', num_classes=10, pretrained=True)
m, _ = model_utils.make_and_restore_model(arch=model, dataset=ds, add_custom_forward=True)
train_loader, val_loader = ds.make_loaders(batch_size=20, workers=4)

# Create a cox store for logging
out_store = cox.store.Store("/content/gdrive/My Drive")

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'lr': 0.01,
    'save_ckpt_iters': 1,
    'attack_lr': 1.5, 
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, datasets.CIFAR)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, datasets.CIFAR)
# Train a model
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)

In [ ]:
ds = datasets.CIFAR(data_path='./data')

model = timm.create_model('ecaresnet26t', num_classes=10, pretrained=True)
m, _ = model_utils.make_and_restore_model(arch=model,resume_path="/content/gdrive/My Drive/AdversarialModel/ecaresnet26t-CIFAR10-Adv.pt", dataset=ds, add_custom_forward=True)

torch_dict = torch.load("/content/gdrive/My Drive/AdversarialModel/ecaresnet26t-CIFAR10-Adv.pt")["model"]
oldnames = []
keysToDelete = []
names = []
for key in torch_dict:
  if "module.model.model." in key:
    oldnames.append(key)
    names.append(key.replace("module.model.model.", ""))
  else:
    keysToDelete.append(key)
for i in range(len(oldnames)):
  torch_dict[names[i]] = torch_dict.pop(oldnames[i])
for key in keysToDelete:
  torch_dict.pop(key)
model.load_state_dict(torch_dict)
train_loader, val_loader = ds.make_loaders(batch_size=20, workers=4)

class InputNormalize(torch.nn.Module):
    '''
    A module (custom layer) for normalizing the input to have a fixed 
    mean and standard deviation (user-specified).
    '''
    def __init__(self, new_mean, new_std):
        super(InputNormalize, self).__init__()
        new_std = new_std[..., None, None]
        new_mean = new_mean[..., None, None]

        self.register_buffer("new_mean", new_mean)
        self.register_buffer("new_std", new_std)

    def forward(self, x):
        x = torch.clamp(x, 0, 1)
        x_normalized = (x - self.new_mean)/self.new_std
        return x_normalized



In [ ]:
import shap
import logging
import numpy as np
batch = next(iter(val_loader))
images,labels = batch
logging.getLogger('shap').setLevel(logging.ERROR)

background = images[:10].to("cuda")
normalizer = InputNormalize(torch.tensor([0.4914, 0.4822, 0.4465]),torch.tensor([0.2023, 0.1994, 0.2010]))
test_images = normalizer(images[10:20])
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_images)
shap_numpy = np.abs([np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]).mean(0)
test_numpy = (np.swapaxes(np.swapaxes(images[10:20].numpy(), 1, -1), 1, 2)  * 255).astype(np.uint8)
shap.plots.image(shap_numpy, test_numpy)

In [ ]:
import shap
import logging
import numpy as np
batch = next(iter(val_loader))
images,labels = batch
logging.getLogger('shap').setLevel(logging.ERROR)

background = images[:15].to("cuda")
normalizer = InputNormalize(torch.tensor([0.4914, 0.4822, 0.4465]),torch.tensor([0.2023, 0.1994, 0.2010]))
test_images = images[15:20].to("cuda")
labels = labels.to("cuda")
kwargs = {
    'constraint':'2', 
    'eps': 0.5,
    'step_size': 1,
    'iterations': 20,
    'do_tqdm': True,
}
_, im_adv = m(test_images, labels[15:20], make_adv=True, **kwargs)
im_adv = im_adv.to("cpu")
test_images = normalizer(im_adv)
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_images)
shap_numpy = np.abs([np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]).mean(0)
test_numpy = (np.swapaxes(np.swapaxes(im_adv.numpy(), 1, -1), 1, 2)  * 255).astype(np.uint8)
shap.plots.image(shap_numpy, test_numpy)